# 构建一个基于图数据库的问答应用

在本指南中，我们将介绍如何在图数据库上创建问答链的基本方法。这些系统将允许我们对图数据库中的数据进行提问，并获得自然语言的答案。

## ⚠️ 安全提示 ⚠️

构建图数据库的问答系统需要执行由模型生成的图查询。这存在一定的风险。请确保您的数据库连接权限始终根据链/代理的需求尽可能缩小范围。这将降低（但不能完全消除）构建模型驱动系统的风险。有关通用安全最佳实践的更多信息，请[点击此处](/docs/security)。

## 架构

从高层次来看，大多数图链的步骤如下：

1. **将问题转换为图数据库查询**：模型将用户输入转换为图数据库查询（例如Cypher）。
2. **执行图数据库查询**：执行图数据库查询。
3. **回答问题**：模型根据查询结果回应用户输入。

![sql_usecase.png](../../static/img/graph_usecase.png)

## 准备工作
#### 安装依赖

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain @langchain/community @langchain/openai @langchain/core neo4j-driver
</Npm2Yarn>
```

#### 设置环境变量

本例中我们将使用OpenAI：

```env
OPENAI_API_KEY=your-api-key

# 可选，使用LangSmith以获得最佳可观测性
LANGSMITH_API_KEY=your-api-key
LANGSMITH_TRACING=true

# 如果不在无服务器环境中，减少追踪延迟
# LANGCHAIN_CALLBACKS_BACKGROUND=true
```

接下来，我们需要定义Neo4j的凭证。
请按照[这些安装步骤](https://neo4j.com/docs/operations-manual/current/installation/)设置Neo4j数据库。

```env
NEO4J_URI="bolt://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="password"
```

以下示例将创建一个与Neo4j数据库的连接，并使用关于电影及其演员的示例数据填充它。

In [ ]:
import "neo4j-driver";
import { Neo4jGraph } from "@langchain/community/graphs/neo4j_graph";

const url = process.env.NEO4J_URI;
const username = process.env.NEO4J_USERNAME;
const password = process.env.NEO4J_PASSWORD;
const graph = await Neo4jGraph.initialize({ url, username, password });

// 导入电影信息
const moviesQuery = `LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))`

await graph.query(moviesQuery);

Schema refreshed successfully.


[]

## 图模式

为了让LLM能够生成Cypher语句，它需要有关图模式的信息。当您实例化一个图对象时，它会检索有关图模式的信息。如果您之后对图进行了任何更改，可以运行`refreshSchema`方法来刷新模式信息。

In [4]:
await graph.refreshSchema()
console.log(graph.getSchema())

Node properties are the following:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING}, Person {name: STRING}, Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre), (:Person)-[:DIRECTED]->(:Movie), (:Person)-[:ACTED_IN]->(:Movie)


很好！我们已经有一个可以查询的图数据库。现在让我们尝试将其连接到LLM。

## 链

让我们使用一个简单的链，它接收一个问题，将其转换为Cypher查询，执行查询，并使用结果回答原始问题。

![graph_chain.webp](../../static/img/graph_chain.webp)

LangChain为此工作流提供了一个内置的链，专为与Neo4j配合使用而设计：`GraphCypherQAChain`。

```{=mdx}
:::warning

本指南中使用的`GraphCypherQAChain`将在提供的数据库上执行Cypher语句。
在生产环境中，请确保数据库连接使用仅包含必要权限的狭域凭证。

否则可能导致数据损坏或丢失，因为调用代码可能会尝试执行删除、数据修改命令，或者在数据库中存在敏感数据的情况下读取敏感数据。

:::
```

In [5]:
import { GraphCypherQAChain } from "langchain/chains/graph_qa/cypher";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-3.5-turbo", temperature: 0 })
const chain = GraphCypherQAChain.fromLLM({
  llm,
  graph,
});
const response = await chain.invoke({ query: "What was the cast of the Casino?" })
console.log(response)

{ result: "James Woods, Joe Pesci, Robert De Niro, Sharon Stone" }

### 后续步骤

对于更复杂的查询生成，我们可能需要创建少量样本提示或添加查询检查步骤。有关此类高级技术和其他技术，请参阅：

* [提示策略](/docs/how_to/graph_prompting)：高级提示工程技巧。
* [值映射](/docs/how_to/graph_mapping)：将问题中的值映射到数据库的技巧。
* [语义层](/docs/how_to/graph_semantic)：实现语义层的技巧。
* [构建图](/docs/how_to/graph_constructing)：构建知识图谱的技巧。
